In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set_theme(style="darkgrid")
import dask.dataframe as dd
import plotly.express as px
import json
import numpy as np
import janitor
import os
import statistics as sts

In [2]:
#definición de funciones compartidas por el Dr. Melesio
def q0(x):
    return x.quantile(0)
def q25(x):
    return x.quantile(.25)
def q50(x):
    return x.quantile(.50)
def q75(x):
    return x.quantile(.75)
def IQR(x):
    return x.quantile(.75) - x.quantile(.25)
def skew(x):
    return np.median(x) - np.mean(x)
def lower_fence(x):
    x = q25(x) - (1.5 * IQR(x))
    
    return np.max([0,x])
def upper_fence(x):
    return q75(x) + (1.5 * IQR(x))

In [3]:
#Se lee el contenido de la base de datos fusionada
inputpath ="/home/reyes/backup/retc20042022fusionada/retc_2004_2022_complete.csv"
joined = pd.read_csv(inputpath, sep=",",low_memory=False, encoding='utf-8')


In [4]:
#Se reemplazan los valores NaN de la columna iarc_group
joined.fillna({"iarc_group":"Without Group"},inplace=True)

In [5]:
#Se listan las columnas de la base de datos
print(joined.columns)

Index(['_id', 'actprincipal', 'actsemarnat', 'anio', 'calle', 'cas',
       'claveambiental', 'codigopostal', 'colonia', 'cve_ent', 'cve_mun',
       'dba_changes', 'dbdescmod', 'dbmod', 'descscian', 'dmsdefault',
       'dmsformat', 'em_agua', 'em_aire', 'em_suelo', 'enentidad', 'enmexico',
       'enmunicipio', 'entrec1', 'entrec2', 'establecimiento', 'estado',
       'finallat', 'finallng', 'gruposustancia', 'iarc_agent', 'iarc_group',
       'iarc_info', 'iarc_volume', 'iarc_year', 'lat', 'latitudnorte', 'lng',
       'localidad', 'longitudoeste', 'municipio', 'nra', 'numexterior',
       'numinterior', 'parqueindustrial', 'scian', 'sector', 'subsector',
       'sustancia', 'tr_alcantarillado', 'tr_coprocesamiento', 'tr_dispfinal',
       'tr_incineracion', 'tr_otros', 'tr_reciclado', 'tr_reutilizacion',
       'tr_tratamiento', 'unidad', 'utmx', 'utmy'],
      dtype='object')


In [23]:
#ejemplo para las estadisticas del agua, como los datos estan en rangos muy dispersos se descartan los valores menores a 1
agua = joined[(joined["em_agua"]> 1) & (joined["anio"]==2022)].sort_values("iarc_group")


In [17]:

#se aplica una escala logaritica base 10
agua["log_em_agua"] = np.log10(agua.em_agua)

In [ ]:
agua["log_em_agua"] = np.log10(agua.em_agua)
print(agua)

In [27]:
agua["cve_ent"].unique().shape

(32,)

In [28]:
fig = px.box(agua,
             x="cve_ent",
             y="log_em_agua",
             color="iarc_group",
             #hover_data=['ENT_NOMBRE','TASA_EST_3_1_100k','SEXO','RANGO_EDAD'],
             labels={'iarc_group':'IARC Group'},
             width=3000,
             height=720)
            #  points='all')
fig.update_traces(quartilemethod='exclusive')
fig.update_layout(
    title="Pollutant releases by State and IARC Group (2022)",
    yaxis_title="Pollutant releases to water in Kg registred by state in log10 scale",
    xaxis_title="State"
)
#fig.write_image("output/boxplot_tasas_estandarizadas_edad_sexo_2022.pdf")
fig.show()

/home/reyes/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## Boxplot por sustancia

### Se realiza una copia de la base de datos

In [33]:
to_analize = joined[[
    "anio", "cve_ent",'estado', 'sustancia', 'iarc_group',
    'em_agua', 'em_aire', 'em_suelo', 
    'tr_alcantarillado', 'tr_coprocesamiento', 'tr_dispfinal', 'tr_incineracion', 'tr_otros', 'tr_reciclado', 'tr_reutilizacion', 'tr_tratamiento',
]]

In [34]:
to_analize.head()

,anio,cve_ent,estado,sustancia,iarc_group,em_agua,em_aire,em_suelo,tr_alcantarillado,tr_coprocesamiento,tr_dispfinal,tr_incineracion,tr_otros,tr_reciclado,tr_reutilizacion,tr_tratamiento
0,2008,14,Jalisco,Cromo (compuestos),3,0.002520,1.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2008,14,Jalisco,Níquel (compuestos),2B,0.006300,6.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2008,15,México,Cromo (compuestos),3,2.061780,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2008,15,México,Mercurio (compuestos),3,0.009818,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2008,15,México,Níquel (compuestos),2B,1.924328,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Se realiza el agrupamiento por anio, cve_ent, sustancia, iarc_group

In [35]:
grouped = to_analize.groupby(['anio','cve_ent','estado','sustancia', 'iarc_group']) \
    .agg({'em_aire':[sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "em_agua": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "em_suelo": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_alcantarillado": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_coprocesamiento": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_dispfinal": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_incineracion": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_otros": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_reciclado": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_reutilizacion": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
            "tr_tratamiento": [sts.mean, 'min', lower_fence, q25, q50, q75, upper_fence, 'max', IQR], \
        }) 

#### Se rellenan los espacios faltantes generados por las agrupaciones

In [36]:

grouped["em_aire"].reset_index()
grouped["em_agua"].reset_index()
grouped["em_suelo"].reset_index()
grouped["tr_alcantarillado"].reset_index()
grouped["tr_coprocesamiento"].reset_index()
grouped["tr_dispfinal"].reset_index()
grouped["tr_incineracion"].reset_index()
grouped["tr_otros"].reset_index()
grouped["tr_reciclado"].reset_index()
grouped["tr_reutilizacion"].reset_index()
grouped["tr_tratamiento"].reset_index()

,anio,cve_ent,estado,sustancia,iarc_group,mean,min,lower_fence,q25,q50,q75,upper_fence,max,IQR
0,2004,1,Aguascalientes,Arsénico,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2004,1,Aguascalientes,Asbesto,*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2004,1,Aguascalientes,Benceno,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2004,1,Aguascalientes,Bióxido de carbono,*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2004,1,Aguascalientes,Cadmio,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13570,2022,32,Zacatecas,Plomo (Compuestos solubles),2B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13571,2022,32,Zacatecas,"Plomo (polvos, respirables, vapores o humos)",2B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13572,2022,32,Zacatecas,Sulfato de cobre,*,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13573,2022,32,Zacatecas,Tolueno,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Se almacennan los resultados

In [37]:
grouped["em_aire"].to_csv("output/g1_subs_em_aire.csv", index=False)
grouped["em_agua"].to_csv("output/g1_subs_em_agua.csv", index=False)
grouped["em_suelo"].to_csv("output/g1_subs_em_suelo.csv", index=False)
grouped["tr_alcantarillado"].to_csv("output/g1_subs_tr_alcantarillado.csv", index=False)
grouped["tr_coprocesamiento"].to_csv("output/g1_subs_tr_coprocesamiento.csv", index=False)
grouped["tr_dispfinal"].to_csv("output/g1_subs_tr_dispfinal.csv", index=False)
grouped["tr_incineracion"].to_csv("output/g1_subs_tr_incineracion.csv", index=False)
grouped["tr_otros"].to_csv("output/g1_subs_tr_otros.csv", index=False)
grouped["tr_reciclado"].to_csv("output/g1_subs_tr_reciclado.csv", index=False)
grouped["tr_reutilizacion"].to_csv("output/g1_subs_tr_reutilizacion.csv", index=False)
grouped["tr_tratamiento"].to_csv("output/g1_subs_tr_tratamiento.csv", index=False)

# New Boxplot

In [44]:
to_bp = joined[[
    "anio", "cve_ent",'estado', 'sustancia', 'iarc_group', 'cas',
    'em_agua', 'em_aire', 'em_suelo', 
    'tr_alcantarillado', 'tr_coprocesamiento', 'tr_dispfinal', 'tr_incineracion', 'tr_otros', 'tr_reciclado', 'tr_reutilizacion', 'tr_tratamiento',
]]

to_bp

,anio,cve_ent,estado,sustancia,iarc_group,cas,em_agua,em_aire,em_suelo,tr_alcantarillado,tr_coprocesamiento,tr_dispfinal,tr_incineracion,tr_otros,tr_reciclado,tr_reutilizacion,tr_tratamiento
0,2008,14,Jalisco,Cromo (compuestos),3,Cr,0.002520,1.950000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2008,14,Jalisco,Níquel (compuestos),2B,Ni,0.006300,6.780000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2008,15,México,Cromo (compuestos),3,Cr,2.061780,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2008,15,México,Mercurio (compuestos),3,Hg,0.009818,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2008,15,México,Níquel (compuestos),2B,Ni,1.924328,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171855,2013,4,Campeche,Bióxido de carbono,*,124-38-9,0.000000,8.870311e+06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171856,2013,14,Jalisco,Bióxido de carbono,*,124-38-9,0.000000,1.722576e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171857,2013,19,Nuevo León,Bióxido de carbono,*,124-38-9,0.000000,3.397210e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
171858,2013,19,Nuevo León,Bióxido de carbono,*,124-38-9,0.000000,2.144423e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
arsenico = to_bp[(to_bp["cas"] == "7440-38-2") | (to_bp["cas"] == "As") | (to_bp["cas"] == "S/C1")].sort_values(by=["anio", "sustancia"])
#arsenico = to_bp[(to_bp["cas"] == "7440-38-2") or (to_bp["cas"] == "As") or (to_bp["cas"] == "S/C1") ].sort_values(by=["anio", "sustancia"])
as_aire = arsenico[(arsenico["em_aire"]>0)]
as_aire["as_aire"] = np.log10(as_aire.em_aire)
#as_agua = arsenico[(arsenico["em_agua"]>0)]

#as_aire["cve_ent"].unique().shape
arsenico.head()



/tmp/ipykernel_7919/1455287301.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,anio,cve_ent,estado,sustancia,iarc_group,cas,em_agua,em_aire,em_suelo,tr_alcantarillado,tr_coprocesamiento,tr_dispfinal,tr_incineracion,tr_otros,tr_reciclado,tr_reutilizacion,tr_tratamiento
115740,2004,15,México,Arsénico,1,7440-38-2,1.200000e-07,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115743,2004,14,Jalisco,Arsénico,1,7440-38-2,2.460000e-03,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115746,2004,1,Aguascalientes,Arsénico,1,7440-38-2,3.132000e-03,0.0,0.0,0.003132,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115748,2004,15,México,Arsénico,1,7440-38-2,1.863000e-06,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
115758,2004,19,Nuevo León,Arsénico,1,7440-38-2,1.200000e-02,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
fig = px.box(as_aire,
             x="cve_ent",
             y="as_aire",
             color="sustancia",
             #hover_data=['ENT_NOMBRE','TASA_EST_3_1_100k','SEXO','RANGO_EDAD'],
             #labels={'iarc_group':'IARC Group'},
             width=1500,
             height=720)
            #  points='all')
fig.update_traces(quartilemethod='exclusive')
fig.update_layout(
    title="Pollutant releases of Arsenic by State (2004-2022)",
    yaxis_title="Pollutant releases to air in Kg registred by state in log10 scale",
    xaxis_title="State"
)
fig.write_image("output/boxplot_emisiones_arsenico_log10_2004_2022.pdf")
fig.show()

/home/reyes/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



## dos

In [77]:
to_bp_1 = joined[[
    "anio", "cve_ent",'estado', 'sustancia', 'iarc_group', 'cas',
    'em_agua', 'em_aire', 'em_suelo', 
    'tr_alcantarillado', 'tr_coprocesamiento', 'tr_dispfinal', 'tr_incineracion', 'tr_otros', 'tr_reciclado', 'tr_reutilizacion', 'tr_tratamiento',
]]

arsenico_2022 = to_bp_1[(to_bp_1["cas"] == "7440-38-2") | (to_bp_1["cas"] == "As") | (to_bp_1["cas"] == "S/C1") & (to_bp_1["anio"]==2022)].sort_values(by=["anio", "sustancia", "cve_ent"])
as_aire_2022 = arsenico_2022[(arsenico_2022["em_aire"]>0)]
#as_aire_2022["as_aire"] = np.log10(as_aire_2022.em_aire)


fig = px.box(as_aire_2022,
             x="cve_ent",
             y="em_aire",
             color="sustancia",
             #hover_data=['ENT_NOMBRE','TASA_EST_3_1_100k','SEXO','RANGO_EDAD'],
             #labels={'iarc_group':'IARC Group'},
             width=1500,
             height=720)
            #  points='all')
fig.update_traces(quartilemethod='exclusive')
fig.update_layout(
    title="Pollutant releases of Arsenic by State (2022)",
    yaxis_title="Pollutant releases of Arsenic to air in Kg registred by state",
    xaxis_title="State"
)
fig.write_image("output/boxplot_emisiones_arsenico_2022.pdf")
fig.write_html("output/boxplot_emisiones_arsenico_2022.html")
fig.show()

/home/reyes/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [80]:
to_bp_1 = joined[[
    "anio", "cve_ent",'estado', 'sustancia', 'iarc_group', 'cas',
    'em_agua', 'em_aire', 'em_suelo', 
    'tr_alcantarillado', 'tr_coprocesamiento', 'tr_dispfinal', 'tr_incineracion', 'tr_otros', 'tr_reciclado', 'tr_reutilizacion', 'tr_tratamiento',
]]

arsenico_2022 = to_bp_1[(to_bp_1["cas"] == "7440-38-2") | (to_bp_1["cas"] == "As") | (to_bp_1["cas"] == "S/C1") & (to_bp_1["anio"]==2022)].sort_values(by=["anio", "sustancia", "cve_ent"])
as_aire_2022 = arsenico_2022[(arsenico_2022["em_aire"]>1)]
as_aire_2022["as_aire"] = np.log10(as_aire_2022.em_aire)


fig = px.box(as_aire_2022,
             x="cve_ent",
             y="as_aire",
             color="sustancia",
             #hover_data=['ENT_NOMBRE','TASA_EST_3_1_100k','SEXO','RANGO_EDAD'],
             #labels={'iarc_group':'IARC Group'},
             width=1500,
             height=720)
            #  points='all')
fig.update_traces(quartilemethod='exclusive')
fig.update_layout(
    title="Pollutant releases of Arsenic by State (2022)",
    yaxis_title="Pollutant releases of Arsenic to air in Kg registred by state (log10 scale)",
    xaxis_title="State"
)
fig.write_image("output/boxplot_emisiones_arsenico_log10_2022.pdf")
fig.write_html("output/boxplot_emisiones_arsenico_log10_2022.html")
fig.show()

/tmp/ipykernel_7919/2825647783.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/reyes/.local/lib/python3.10/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

